In [18]:
from pymongo import MongoClient
import os
import pandas as pd
from neo4j import GraphDatabase

In [9]:
mongo = MongoClient(os.getenv("MONGO_URI", "mongodb://localhost:27017"))

In [10]:
mdb = mongo["backoffice"]

In [17]:

def import_mongo():
    df_prov = pd.read_csv("data/proveedor.csv",encoding="Windows 1252",sep=";")
    df_tel  = pd.read_csv("data/telefono.csv",encoding="Windows 1252",sep=";")
    df_prod = pd.read_csv("data/producto.csv",encoding="Windows 1252",sep=";")
    df_op   = pd.read_csv("data/op.csv",encoding="Windows 1252",sep=";")
    df_det  = pd.read_csv("data/detalle_op.csv",encoding="Windows 1252",sep=";")

    mdb.proveedores.delete_many({})
    mdb.proveedores.insert_many(df_prov.to_dict("records"))

    mdb.telefonos.delete_many({})
    mdb.telefonos.insert_many(df_tel.to_dict("records"))

    mdb.productos.delete_many({})
    mdb.productos.insert_many(df_prod.to_dict("records"))

    mdb.ordenes.delete_many({})
    mdb.ordenes.insert_many(df_op.to_dict("records"))

    mdb.detalles.delete_many({})
    mdb.detalles.insert_many(df_det.to_dict("records"))
import_mongo()

In [21]:
uri = os.getenv("NEO4J_URI", "bolt://localhost:7687")
neo = GraphDatabase.driver(uri, auth=("neo4j", os.getenv("NEO4J_PASSWORD", "epicpassword123")))

In [30]:
def import_neo4j():
    with neo.session() as sess:
        sess.run("MATCH (n) DETACH DELETE n")
        # proveedores
        df = pd.read_csv("data/proveedor.csv",encoding="Windows 1252",sep=";")
        for _,r in df.iterrows():
            sess.run("""
                CREATE (p:Proveedor {
                  id_proveedor:$id_proveedor,
                  cuit: $CUIT_proveedor,
                  razon_social: $razon_social,
                  activo: $activo,
                  habilitado: $habilitado
                })
            """, **r.to_dict())

        # telefonos
        df = pd.read_csv("data/telefono.csv",encoding="Windows 1252",sep=";")
        for _,r in df.iterrows():
            sess.run("""
                MATCH (p:Proveedor {id_proveedor: $id_proveedor})
                CREATE (p)-[:TIENE_TELEFONO {numero:$telefono}]->(:Telefono)
            """, id_proveedor=r.id_proveedor, telefono=r.nro_telefono)

        # productos
        df = pd.read_csv("data/producto.csv",encoding="Windows 1252",sep=";")
        for _,r in df.iterrows():
            sess.run("""
                CREATE (:Producto {
                  codigo: $id_producto,
                  nombre: $descripcion,
                  marca: $marca,
                  precio: $precio
                })
            """, **r.to_dict())

        # ordenes y detalles
        df_op = pd.read_csv("data/op.csv",encoding="Windows 1252",sep=";")
        df_op["fecha"] = pd.to_datetime(df_op["fecha"], dayfirst=True).dt.strftime("%Y-%m-%d")
        df_det= pd.read_csv("data/detalle_op.csv",encoding="Windows 1252",sep=";")
        for _,o in df_op.iterrows():
            sess.run("""
                CREATE (o:Orden {
                     id_pedido:$id_pedido,
                     id_proveedor:$id_proveedor,
                  fecha: date($fecha)
                })
            """, **o.to_dict())
        for _,d in df_det.iterrows():
            sess.run("""
                MATCH (o:Orden {id_pedido:$id_pedido,id_producto:$id_producto}), (pr:Producto {codigo:$id_producto})
                CREATE (o)-[:CONTiene {cantidad:$cantidad}]->(pr)
            """, **d.to_dict())
import_neo4j()